### Extract SEC Filing Archives

In [ ]:
import os

def create_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
import zipfile

def extract_archive(archive, directory):
    with zipfile.ZipFile(archive, 'r') as zip_ref:
        zip_ref.extractall(directory)

In [ ]:
import os
import progressbar

sec_dir = 'sec_filings'
for item in progressbar.progressbar(sorted(os.listdir(f'{sec_dir}/zips'))):
    if item.startswith('20'):
        quarter = item.split('.')[0]
        quarter_directory = f'{sec_dir}/{quarter}'
        create_dir(quarter_directory)
        extract_archive(f'{sec_dir}/zips/{item}', quarter_directory)

### Extract Fundamental Data

In [1]:
try:
    import cPickle as pickle
except:
    import pickle
import re
from requests import get
from collections import defaultdict
import time
import progressbar
import pandas as pd
import numpy as np

symbols_df = pd.read_csv('ITOT_holdings.csv')
symbols = symbols_df.loc[symbols_df.market_cap.str.replace(',', '').astype(np.float).between(10000,500000)].symbol.values
URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
CIK_RE = re.compile(r'.*CIK=(\d{10}).*')

cik_dict = {}
for ticker in progressbar.progressbar(symbols):
    #time.sleep(.1)
    results = CIK_RE.findall(get(URL.format(ticker)).content.decode("utf-8") )
    if len(results):
        cik_dict[str(results[0])] = str(ticker).lower()
f = open('cik_dict_micro_cap.pickle', 'wb')
pickle.dump(cik_dict, f)
f.close()

100% (1222 of 1222) |####################| Elapsed Time: 0:04:52 Time:  0:04:52


In [2]:
import pandas as pd
import os
import progressbar
import pickle
import numpy as np

def propagate_latest(sym_df):
    sym_df = sym_df.sort_values('filed')
    sym_df = sym_df.fillna(method='ffill')
    return sym_df

cik_dict = pickle.load(open('cik_dict_micro_cap.pickle', 'rb'))
sec_dir = 'sec_filings'
values = ['CommonStockSharesOutstanding', 'EarningsPerShareBasic', 'EarningsPerShareBasicAndDiluted']
quarters = [
    quarter
    for quarter in os.listdir(sec_dir)
    if quarter.startswith('20')
]
df_list = []
used_quarters = []
for quarter in progressbar.progressbar(quarters): # progressbar.progressbar(quarters):  ['2020q3']
    sub_df = pd.read_csv(f'{sec_dir}/{quarter}/sub.txt', delimiter='\t')
    if sub_df.shape[0] == 0:
        continue
    num_df = pd.read_csv(f'{sec_dir}/{quarter}/num.txt', delimiter='\t')
    
    # need to understand how ddate works here
    num_df = num_df.sort_values('ddate')
    num_df = num_df[['adsh', 'tag', 'ddate', 'value']].drop_duplicates(
        subset=['adsh', 'tag'], keep='last'
    )
    #sub_df.loc[:, 'symbol'] = sub_df.instance.apply(
    #    lambda x: x.split('-')[0].upper()
    #)
    sub_df.loc[:, 'symbol'] = sub_df.apply(
        lambda x: cik_dict[f'{x.cik:010}'].upper() if f'{x.cik:010}' in cik_dict else x.instance.split('-')[0].upper(),
        axis=1
    )
    sub_df.loc[:, 'year'] = int(quarter[:4])
    sub_df.loc[:, 'quarter'] = int(quarter[5])
    for index, value in enumerate(values):
        value_df = num_df.loc[num_df.tag == value][['adsh', 'value']]
        value_df = value_df.rename(columns={"value": value})
        sub_df = sub_df.merge(
            value_df, on='adsh',
            how='left'
        )
    sub_df = sub_df.groupby('symbol').apply(propagate_latest).reset_index(drop=True)
    sub_df = sub_df.sort_values('filed').drop_duplicates(subset=['symbol'], keep='last') #duplicate adsh values found
    sub_df.loc[:, 'EarningsPerShareBasic'] = np.fmin(sub_df.EarningsPerShareBasic, sub_df.EarningsPerShareBasicAndDiluted)
    df_list.append(sub_df[values + ['symbol', 'year', 'quarter']])
    used_quarters.append(quarter)
fundamental_df = pd.concat(df_list, axis=0, ignore_index=True)
fundamental_df.to_parquet('fundamentals_micro.parquet')

 23% (11 of 47) |#####                   | Elapsed Time: 0:05:13 ETA:   0:01:22/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 74% (35 of 47) |#################       | Elapsed Time: 0:18:47 ETA:   0:08:04/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100% (47 of 47) |########################| Elapsed Time: 0:26:17 Time:  0:26:17


In [ ]:
sub_df.loc[sub_df.cik == 789570][['adsh', 'filed']]

In [ ]:
sub_df.iloc[3797]

In [ ]:
cik_dict[f'{1372612:010}']

In [ ]:
f'{789570:010}'

In [ ]:
cik_dict.items()

In [ ]:
sub_df.

In [ ]:
sub_df.shape

In [ ]:
sorted(num_df.loc[num_df.adsh == '0001564590-20-042654'].tag.unique())

In [ ]:
num_df.columns

In [ ]:
sub_df.loc[sub_df.symbol == 'AAPL']

In [ ]:
import numpy as np
np.fmin(sub_df.EarningsPerShareBasic, sub_df.EarningsPerShareBasicAndDiluted)